# Global Shark Attacks

- note
- note
- note
-
-


## Import Libraries

In [61]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
from scipy.stats.mstats import winsorize

import math
from sklearn.preprocessing import OneHotEncoder, Normalizer, LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.neighbors import KNeighborsRegressor

import warnings
warnings.filterwarnings('ignore')

## Import Data

In [62]:
data = pd.read_csv('attacks.csv',encoding='latin-1')

## First Review and Clean Data

In [63]:
data.shape

(25723, 24)

In [64]:
data.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


### Cleaning Columns

In [29]:
for col in data:
    print(data[col].value_counts(), '\n')

# replace Case Numer with unique identifier
# extract month and year from date (where possible)
# clean year to identical format or extract from date
# check if years are the same from date and year
# Type ??????? what do those categories mean and are they actually reliable? probaly objective call
# check country values if there are somekind of double counts?
# Area? what to do with the Area? probably has a connection to the kind of shark
# Location? what to do with Location? probably has a connection to the kind of shark
# Activity - Cleaning down to a few categories and sum the others under unknown
# name . drop
# sex
# age
# injury
# fatal

0               2400
1990.05.10         2
1920.00.00.b       2
2012.09.02.b       2
1966.12.26         2
                ... 
1888.01.22         1
1974.03.21         1
1960.08.04.b       1
1905.03.25         1
1949.01.13         1
Name: Case Number, Length: 6287, dtype: int64 

1957           11
1942            9
1956            8
1950            7
1958            7
               ..
06-Mar-2004     1
02-Feb-1968     1
13-Jul-2006     1
17-Nov-2002     1
Jan-1936        1
Name: Date, Length: 5433, dtype: int64 

2015.0    143
2017.0    136
2016.0    130
2011.0    128
2014.0    127
         ... 
1753.0      1
77.0        1
1785.0      1
1580.0      1
1543.0      1
Name: Year, Length: 249, dtype: int64 

Unprovoked      4595
Provoked         574
Invalid          547
Sea Disaster     239
Boating          203
Boat             137
Questionable       2
Boatomg            1
Name: Type, dtype: int64 

USA                           2229
AUSTRALIA                     1338
SOUTH AFRICA           

Name: href formula, Length: 6290, dtype: int64 

http://sharkattackfile.net/spreadsheets/pdf_directory/w014.01.25-Grant.pdf                  4
http://sharkattackfile.net/spreadsheets/pdf_directory/2014.10.02.b-Vandenberg.pdf           3
http://sharkattackfile.net/spreadsheets/pdf_directory/1907.10.16.R-HongKong.pdf             2
http://sharkattackfile.net/spreadsheets/pdf_directory/1935.06.05.R-SolomonIslands.pdf       2
http://sharkattackfile.net/spreadsheets/pdf_directory/1931.09.21.a-b-Holaday-Barrows.pdf    2
                                                                                           ..
http://sharkattackfile.net/spreadsheets/pdf_directory/1959.08.11-Tsuchiya.pdf               1
http://sharkattackfile.net/spreadsheets/pdf_directory/1957.07.09-Kris.pdf                   1
http://sharkattackfile.net/spreadsheets/pdf_directory/1955.08.30.a-DaleStrand.pdf           1
http://sharkattackfile.net/spreadsheets/pdf_directory/2013.06.14.R-Foreman.pdf              1
http://shar

After a first brief review of the data it becomes clear very fast that this data will require a lot of data cleaning which is why I will review column by columns.

#### Clean Column Names

In [65]:
data.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

**Changes needed:** <br>
lower case <br>
underscore '_' <br>
drop empty spaces ' ', <br>
replace '.' with underscore '_' <br>
remove (Y/N)

In [68]:
data.columns = [column.lower().replace('.', '').replace(' ','_').replace(':','').replace('(y/n)','') for column in data.columns]

In [69]:
data.columns

Index(['case_number', 'date', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'sex_', 'age', 'injury', 'fatal_', 'time',
       'species_', 'investigator_or_source', 'pdf', 'href_formula', 'href',
       'case_number1', 'case_number2', 'original_order', 'unnamed_22',
       'unnamed_23'],
      dtype='object')

#### case_number, case_number1, case_number2

In [71]:
cases = data[['date', 'year', 'case_number', 'case_number1', 'case_number2']].copy()

In [73]:
cases.head()

,date,year,case_number,case_number1,case_number2
0,25-Jun-2018,2018.0,2018.06.25,2018.06.25,2018.06.25
1,18-Jun-2018,2018.0,2018.06.18,2018.06.18,2018.06.18
2,09-Jun-2018,2018.0,2018.06.09,2018.06.09,2018.06.09
3,08-Jun-2018,2018.0,2018.06.08,2018.06.08,2018.06.08
4,04-Jun-2018,2018.0,2018.06.04,2018.06.04,2018.06.04


In [74]:
for col in cases:
    print(cases[col].value_counts(), '\n')

1957           11
1942            9
1956            8
1950            7
1958            7
               ..
06-Mar-2004     1
02-Feb-1968     1
13-Jul-2006     1
17-Nov-2002     1
Jan-1936        1
Name: date, Length: 5433, dtype: int64 

2015.0    143
2017.0    136
2016.0    130
2011.0    128
2014.0    127
         ... 
1753.0      1
77.0        1
1785.0      1
1580.0      1
1543.0      1
Name: year, Length: 249, dtype: int64 

0               2400
1990.05.10         2
1920.00.00.b       2
2012.09.02.b       2
1966.12.26         2
                ... 
1888.01.22         1
1974.03.21         1
1960.08.04.b       1
1905.03.25         1
1949.01.13         1
Name: case_number, Length: 6287, dtype: int64 

1990.05.10        2
1980.07.00        2
1915.07.06.a.R    2
1913.08.27.R      2
2005.04.06        2
                 ..
2014.08.09.R      1
2013.12.16        1
1888.01.22        1
1974.03.21        1
1949.01.13        1
Name: case_number1, Length: 6285, dtype: int64 

2013.10.05      2
1

I drop the case numbers and work with a unique index instead

#### date

#### year

#### drop columns

In [ ]:
to_drop = ['case_number',
            'case_number1',
            'case_number2']

df.drop(to_drop, inplace=True, axis=1)

df.drop(columns=to_drop, inplace=True)

In [28]:
data['Case Number'].is_unique

False

In [24]:
data.shape

(25723, 24)

In [25]:
data.dtypes

Case Number                object
Date                       object
Year                      float64
Type                       object
Country                    object
Area                       object
Location                   object
Activity                   object
Name                       object
Sex                        object
Age                        object
Injury                     object
Fatal (Y/N)                object
Time                       object
Species                    object
Investigator or Source     object
pdf                        object
href formula               object
href                       object
Case Number.1              object
Case Number.2              object
original order            float64
Unnamed: 22                object
Unnamed: 23                object
dtype: object